In [54]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Marketing Analytics Exploratory/ Statistical Analysis Task

## Introduction
As a marketing analyst and you've been told by the Chief Marketing Officer that recent marketing campaigns have not been as effective as they were expected to be. You need to analyse the data set to understand this problem and propose data-driven solution.

## Section 01: Exploratory Data Analysis
1. Are there any null values or outliers?
2. Are there any variables that warrant transformations?
3. Are there any useful variables that we can engineer with the given data?
4. Any patterns or anomalies in the data?

## Section 02: Statistical Analysis
Run statistical tests in the form of regression to answer the questions and propose a data-driven action to the CMO. Interpret the result with non-statistical jargon so that CMO can understand the findings.
1. What factors are significantly related to the number of store purchases?
2. Does US fare significantly better than the rest of the world in terms of total purchases?
3. The supervisor insists that people who buy gold are more conservative. Therefore, people who spent an above average amount on gold in the   last 2 years would have more in store purchases. Justify or refute this statement using an appropriate statistical test.
4. Fish has Omega 3 fatty acids which are good for the brain. Accordingly do "Married PhD candidates" have a significant relation with the amount spend on fish products? What factors are significantly related to amount spend on fish?
5. Is there a significant relationship between geographical regional and success of a campaign?

## Section 03: Data Visualisation
1. Which marketing campaign is most successful?
2. What does the average customer look like for this company?
3. Which products are performing best?
4. Which channels are underperforming?

Loading the dataset

In [55]:
#loading required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#pd.set_option('display.max_columns', None)
#%matplotlib inline
#sns.set_context('notebook')
#sns.set_style('whitegrid')
#sns.set_palette('Blues_r')

#turning off warnings
import warnings
warnings.filterwarnings('ignore')

#importing dataset
df=pd.read_csv("../input/marketing-data/marketing_data.csv")
print(df.info())
df.head()

#### Cleaning the dataset and transforming Income to float

In [56]:
# clean up column names that contain whitespace
df.columns = df.columns.str.replace(' ', '')

# transform Income column to a numerical
df['Income'] = df['Income'].str.replace('$', '')
df['Income'] = df['Income'].str.replace(',', '').astype('float')

#### The cleaned dataset:

In [57]:
df.head()

## **SECTION 1: EXPLORATORY DATA ANALYSIS**

#### Are there any Null values or outliers?

In [58]:
df.isnull().sum().sort_values(ascending=False)

In [59]:
#Plotting 'Income' to get an idea of distribution 
sns.histplot(df['Income'],kde=False);

In [60]:
sns.boxplot(df['Income']);

#### Findings about the income:
* Income contains 24 null values
* It is distributed between `$ 0- $ 100,000` and has some outliers

* Fill null values with the median value to avoid the effect of outliers

In [61]:
df['Income']=df['Income'].fillna(df['Income'].median())

In [62]:
df_to_plot=df.drop(columns=['ID','AcceptedCmp1', 'AcceptedCmp2', 
                            'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 
                            'Response', 'Complain']).select_dtypes(include=np.number)

#### Outliers
Identifying features that contain outliers

In [63]:
df_to_plot.plot(subplots=True, layout=(4,4), kind='box', figsize=(12,14), patch_artist=True);
plt.subplots_adjust(wspace=0.5);

#### Findings about outliers:
Multiple features contain outliers as show in the boxplots above but the only one that suggest wrong data-entry is Year_Birth<=1900.

Removing rows where Year_Birth<=1900.

In [64]:
df=df[df['Year_Birth']>1900].reset_index(drop=True)
plt.figure(figsize=(3,4))
df['Year_Birth'].plot(kind='box', patch_artist=True);

### Are there any variables that need transformation?

In [65]:
df.info()

The variable Dt_Customer represents Date of customer's enrollment with the company so this should be in DateTime format.

In [66]:
df['Dt_Customer']=pd.to_datetime(df['Dt_Customer'])

### Are there any useful variables that we can engineer with the given data?
* The total number of dependents ('Dependents') can be engineered with the sum of 'Kidhome' and 'Teenhome'
* Variable 'Year_Customer' which is year of becoming a customer can be engineered from 'Dt_Customer'
* The total amount spent ('TotalMnt') can be engineered from the sum of all features containing the keyword 'Mnt'
* Similarly the total purchases ('TotalPurchases') can be engineered from the sum of all features containing the keyword 'Purchases'
* The total number of campains accepted ('TotalCampaignsAcc') can be engineered from the sum of all features containing the keywords 'Cmp' and 'Response' (the latest campaign)

In [67]:
list(df.columns)

In [68]:
#performing feature engineering
df['Dependents']=df['Kidhome']+df['Teenhome']

#year becoming a customer
df['Year_Customer']=pd.DatetimeIndex(df['Dt_Customer']).year

#total amount spend
mnt_cols= [col for col in df.columns if 'Mnt' in col]
df['TotalMnt']=df[mnt_cols].sum(axis=1)

#total purchases
purchases_cols=[col for col in df.columns if 'Purchase' in col]
df['TotalPurchases']=df[purchases_cols].sum(axis=1)

#total campaigns accepted
campaigns_cols=[col for col in df.columns if 'Cmp' in col]+['Response']
df['TotalCampaignsAcc']=df[campaigns_cols].sum(axis=1)

#view new features, by customer ID
df[['ID','Dependents', 'Year_Customer', 'TotalMnt', 'TotalPurchases', 'TotalCampaignsAcc']].head()

### Is there any pattern or anomalies in the data? Can we plot them?
* To identify paterns, we will first identify feature correlation. We will use the clustermap below for that; where dark blue means positive correlation and white represents negative correlation.
* From this heatmap we can observe the following clusters of correlated features:
    * "**High Income**" cluster:
        ** Amount spent ('TotalMnt' and other 'Mnt' features) and number of purchases ('TotalPurchases' and other 'Num...Purchases' features) are positively related with 'Income'.
        ** Purchasing in store, on the web, or via the catalog ('NumStorePurchases','NumWebPurchases','NumCatalogPurchases') is positively correlated with 'Income'
   * "**Dependents**"cluster:
       ** Amount spent and number of purchases are negatively correlated with 'Dependents'.
       ** Purchasing deals ('NumDealsPurchases') is positively correlated with 'Dependents' and negatively correlated with 'Income' 
   * "**Advertising Campaigns**" cluster:
       ** Acceptence of advertising campaigns ('AcceptedCmp' and 'Response') are strongly positively correlated with each other.
       ** Weak positive correlation of the advertising campaigns is seen with the "High Income" cluster, and weak negative correlation is seen with the "Have Kids & Teens" cluster
* Anomalies:
    * The number of website visits in the last month ('NumWebVisitsMonth') does not correlate with an increased number of web purchases ('NumWebPurchases')
    * Instead, 'NumWebVisitsMonth' is positively correlated with the number of deals purchased ('NumDealsPurchases'), suggeting that deals are effective way of stiimulating purchase on the website

In [69]:
#calculate correlation matrix
corrs=df.drop(columns='ID').select_dtypes(include=np.number).corr(method='kendall')

sns.clustermap(corrs, cmap="YlGnBu");

#### Plot illustrating the effect of income on spending
 Note: For the purpose of this plot limiting income to < 200,000 to remove outliers

In [70]:
sns.lmplot(x='Income', y='TotalMnt', data=df[df['Income']<200000]);
plt.xticks(rotation=45, horizontalalignment='right');

#### Plot illustrating negative effect of having dependents (kids & teens) on spending:

In [71]:
plt.figure(figsize=(4,4))
sns.boxplot(x='Dependents', y='TotalMnt', data=df);

#### Plot illustrating positive effect of having dependents (kids & teens) on number of deals purchased:

In [72]:
plt.figure(figsize=(4,4))
sns.boxplot(x='Dependents', y='NumDealsPurchases', data=df);

#### Plots illustrating the positive effect of income and negative effect of having kids & teens on advertising campaign acceptance:
NOTE: For the purpose of this graph limiting income to < 200,000 to remove outliers

In [73]:
plt.figure(figsize=(5.5,4));
sns.boxplot(x='TotalCampaignsAcc', y='Income', data=df[df['Income']<200000]);

In [74]:
plt.figure(figsize=(5.5,4))
sns.boxplot(x='TotalCampaignsAcc', y='Dependents', data=df);

#### Investing Anomaly:
* Number of web visits is not positively correlated with the number of web purchases.
* Instead, it is positively correlated with the number of deals purchased.

In [75]:
sns.lmplot(x='NumWebVisitsMonth', y='NumWebPurchases', data=df);

In [76]:
sns.lmplot(x='NumWebVisitsMonth', y='NumDealsPurchases', data=df);

## Section 02: Statistical Analysis
Run statistical tests in the form of regression to answer the questions and propose a data-driven action to the CMO. Interpret the result with non-statistical jargon so that CMO can understand the findings.

### What factors are significantly related to the number of store purchases?
* We will use a Linear Regression model with NumStorePurchases as the target variables, and then use machine learning explainability techniques to get insights about which features predict the number of store purchases
* Begin by plotting the target variable:

In [77]:
sns.countplot(df['NumStorePurchases']);

* Drop uninformative features:
    * ID in unique to each customer
    * Dt_Customer will be dropped in favour of using engineered variable Year_Customer
* Perform one-hot encoding of categorical features.

In [78]:
#drop unique ID
df.drop(columns=['ID', 'Dt_Customer'], inplace=True)

In [79]:
#one hot encoding for categorical features
from sklearn.preprocessing import OneHotEncoder

cat=df.select_dtypes(exclude=np.number)
print("Number of unique values per categorical features:\n", cat.nunique())

#use one hot encoder
enc=OneHotEncoder(sparse=False).fit(cat)
cat_encoded=pd.DataFrame(enc.transform(cat))
cat_encoded.columns=enc.get_feature_names(cat.columns)

#merge with numeric data
num=df.drop(columns=cat.columns)
df2=pd.concat([cat_encoded,num],axis=1)
df2.head()

* Fit linear regression model on training data (70% of dataset)
* Evaluate predictions on test data (30% of dataset) using RMSE

In [80]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

X=df2.drop(columns='NumStorePurchases')
y=df2['NumStorePurchases']

X_train,X_test, y_train,y_test=train_test_split(X,y, test_size=0.3, random_state=1)

model=LinearRegression()
model.fit(X_train,y_train)

preds=model.predict(X_test)
print("Linear Regression Model RMSE:", np.sqrt(mean_squared_error(y_test,preds)))
print("Median value of target variable:", y.median())

The RMSE is exceedingly small compared to the median of the target variable, indicating good model predictions

* Interpreting the results using Permutation importance:
    * Significant feature are:
        * 'TotalPurchases', 'NumCatalogPurchases', 'NumWebPurchases', 'NumDealsPurchases'
        * All other features are not significant

In [81]:
import eli5
from eli5.sklearn import PermutationImportance

perm=PermutationImportance(model, random_state=1).fit(X_test,y_test)
eli5.show_weights(perm,feature_names=X_test.columns.tolist(),top=5)

* Explore the directionality of these effects, using SHAP values:


In [82]:
import shap

#calculate shap values
ex=shap.Explainer(model, X_train)
shap_values=ex(X_test)

#plot
plt.title('SHAP summary for NumStorePurchases',size=16)
shap.plots.beeswarm(shap_values,max_display=4);

* Findings:
    * The number of store purchases increases with higher number of total purchases
    * The number of store purchases decreases with higher number of catalog, web or deals purchases
* Interpretation:
    * Customers who shop the most in the stores are those who shop less via the catalog, website or special deals

### Does US fare significantly better than the Rest of the World in terms of total purchase?
* Plot total number of purchases by country:
   

In [83]:
df.groupby('Country')['TotalPurchases'].sum().sort_values(ascending=False).plot(kind='bar')
plt.title('Total Number of Purchases by Country', size=16)
plt.ylabel('Number of purchases');

* Findings :
    * Spain (SP) has the highest number of purchases
    * US is second to last, therefore the US does not fare better than the Rest of the World in terms of total number of purchases

* Plot total **amount spent** by country:
    * Findings:
        * Spain (SP) has the highest total amount spent on purchases
        * US is second to last, thus the US does not fare better than the rest of the world in terms of the total amount spent on purchases

In [84]:
plt.figure(figsize=(5,4))
df.groupby('Country')['TotalMnt'].sum().sort_values(ascending=False).plot(kind='bar')
plt.title('Total Amount Spent by Country', size=16)
plt.ylabel('Amount Spent');

#### Next Task: The supervisor insists that people who buy gold are more conservative. Therefore, people who spent an above average amount on gold in the last 2 years would have more in store purchases. Justify or refute this statement using an appropriate statistical test
* Plot relationship between amount spent on gold in the last 2 years (MntGoldProds) and number of in store purchases (NumStorePurchases):
    

In [85]:
sns.lmplot(x='MntGoldProds', y='NumStorePurchases', data=df);

* Findings: There is a positive relationship, but is it statistically significant? 
* Perform Kendall Correlation Analysis (non parametric test since MntGoldProducts is not normally distributed and contains outliers):
    * Findings: There is a significant positive correlation between the two variables.

In [86]:
from scipy.stats import kendalltau

kendall_corr = kendalltau(x=df['MntGoldProds'], y=df['NumStorePurchases'])

# print results
print('Kendall correlation (tau): ', kendall_corr.correlation)
print('Kendall p-value: ', kendall_corr.pvalue)

#### Fish has Omega 3 fatty acids which are good for the brain. Accordingly, do "Married PhD candidates" have a significant relation with amount spent on fish?
* To check this claim we will compare amount spent on Fish Products (MntFishProducts) between MarriedPhD candidates and all other customer:
    * Findings: Married PhD candidates spend significantly less on fish products comapared to other customers.

In [87]:
#sum the marital status and phd dummy variables-the Married+PhD will have value of 2
df2['Married_PhD']=df2['Marital_Status_Married']+df2['Education_PhD']
df2['Married_PhD']=df2['Married_PhD'].replace({2:'Married-PhD', 1:'Other', 0:'Other'})

plt.figure(figsize=(2.5,4))
sns.boxplot(x='Married_PhD', y='MntFishProducts', data=df2);

In [88]:
# independent t-test p-value
from scipy.stats import ttest_ind
pval = ttest_ind(df2[df2['Married_PhD'] == 'Married-PhD']['MntFishProducts'], df2[df2['Married_PhD'] == 'Other']['MntFishProducts']).pvalue
print("t-test p-value: ", round(pval, 3))

In [89]:
# now drop the married-phD column created above, to include only the original variables in the analysis below
df2.drop(columns='Married_PhD', inplace=True)

### What other factors are significantly related to the amount spent on fish?
* Like the analysis of NumStorePurchases above, we will use Linear Regression model with MntFishProducts as the target variable, and then use machine learning explainability techniques to get insights about which features predict the amount spent on fish.
* Begin by plotting the target variable:


In [90]:
plt.figure(figsize=(8,4))
sns.distplot(df['MntFishProducts'], kde=False, hist=True, bins=12);
plt.title('MntFishProducts distribution', size=16)
plt.ylabel('count');

* Fit linear regression to training data
* Evaluate predictions on the test data using RMSE
    * The RMSE is excedingly small comapared to the median value of the target variable, indicating good model predictions

In [91]:
X=df2.drop(columns='MntFishProducts')
y=df2['MntFishProducts']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

model=LinearRegression()
model.fit(X_train,y_train)
preds=model.predict(X_test)

# evaluate model using RMSE
print("Linear regression model RMSE: ", np.sqrt(mean_squared_error(y_test, preds)))
print("Median value of target variable: ", y.median())

* Identify features that significantly affect the amount spent on fish, using permutation importance:
    * Significant features:
        * 'TotalMnt', 'MntWines', 'MntMeatProducts', 'MntGoldProds', 'MntSweetProducts', 'MntFruits'
    * All other features are not significant

In [92]:
perm = PermutationImportance(model, random_state=1).fit(X_test, y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist(), top=7)

* Explore the directionality of these effects, using SHAP values:
    * Findings:
        * The amount spent on fish increases with higher total amount spent ('TotalMnt')
        * The amount spent on fish decreases with higher amounts spent on wine, meat, gold, fruit, or sweets ('MntWines', 'MntMeatProducts', 'MntGoldProds', 'MntSweetProducts', 'MntFruits')
    * Interpretation:
        * Customers who spend the most on fish are those who spend less on other products (wine, meat, gold, fruit, and sweets)

In [93]:
import shap

# calculate shap values 
ex = shap.Explainer(model, X_train)
shap_values = ex(X_test)

# plot
plt.title('SHAP summary for MntFishProducts', size=16)
shap.plots.beeswarm(shap_values, max_display=7);

#### Is there a significant relationship between geographical regional and success of a campaign?
* Plot success of campaigns by region:
* Findings:
    * The campaign acceptance rates are low overall
    * The campaign with the highest overall acceptance rate is the most recent campaign (column name: Response)
    * The country with the highest acceptance rate in any campaign is Mexico
   

In [94]:
# convert country codes to correct nomenclature for choropleth plot
# the dataset doesn't provide information about country codes
## ...so I'm taking my best guess about the largest nations that make sense given the codes provided
df['Country_code'] = df['Country'].replace({'SP': 'ESP', 'CA': 'CAN', 'US': 'USA', 'SA': 'ZAF', 'ME': 'MEX'})

# success of campaigns by country code
df_cam = df[['Country_code', 'AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'Response']].melt(
    id_vars='Country_code', var_name='Campaign', value_name='Accepted (%)')
df_cam = pd.DataFrame(df_cam.groupby(['Country_code', 'Campaign'])['Accepted (%)'].mean()*100).reset_index(drop=False)

# rename the campaign variables so they're easier to interpret
df_cam['Campaign'] = df_cam['Campaign'].replace({'AcceptedCmp1': '1',
                                                'AcceptedCmp2': '2',
                                                'AcceptedCmp3': '3',
                                                'AcceptedCmp4': '4',
                                                'AcceptedCmp5': '5',
                                                 'Response': 'Most recent'
                                                })

# choropleth plot
import plotly.express as px

fig = px.choropleth(df_cam, locationmode='ISO-3', color='Accepted (%)', facet_col='Campaign', facet_col_wrap=2,
                    facet_row_spacing=0.05, facet_col_spacing=0.01, width=700,
                    locations='Country_code', projection='natural earth', title='Advertising Campaign Success Rate by Country'
                   )
fig.show()

## SECTION 3: Data Visualisation


#### Which marketing campaign is most successful?
* Plot marketing campaign overall acceptance rates:
    

In [95]:
cam_success=pd.DataFrame(df[['AcceptedCmp1','AcceptedCmp2','AcceptedCmp3','AcceptedCmp4','AcceptedCmp5','Response']].mean()*100, columns=['Percent']).reset_index()

#plot
sns.barplot(x="Percent", y='index', data=cam_success.sort_values('Percent'),palette='Blues');
plt.xlabel('Accepted(%)');
plt.ylabel('Campaign');
plt.title('Marketing campaign success rate',size=16);

* Findings: 
    * The most successful campaign is the most recent ('Response')

### What does the average customer look like for the company?
* Basic demographics:
    * Born in 1969
    * Became a customer in 2013
    * Has an income of around` $ 52,000 per year `
    * Has 1 dependent (roughly split between kids or teens)
    * Made a purchase from our company in the last 49 days

In [96]:
#lists of cols with binary responses
binary_cols=[col for col in df.columns if 'Accepted' in col] + ['Response', 'Complain']

#lists of cols for spending
mnt_cols=[col for col in df.columns if 'Mnt' in col]

#lists of cols for channels
channel_cols=[col for col in df.columns if 'Num' in col] + ['TotalPurchases', 'TotalCampaignsAcc']

In [97]:
#average customer demographics
demographics=pd.DataFrame(round(df.drop(columns=binary_cols+mnt_cols+channel_cols).mean(),1),columns=['Average']).reindex(['Year_Birth','Year_Customer','Income','Dependents','Kidhome','Teenhome', 'Recency'])

demographics

### Which products are performing best?
* The average customer spent...
    * ` $` 25-50 on Fruits, Sweets, Fish, or Gold products
    * Over `$` 160 on Meat products
    * Over `$` 300 on Wines
    * Over `$` 600 total
    * Products performing best:
    * Wines
    * Followed by meats

In [98]:
spending=pd.DataFrame(round(df[mnt_cols].mean(),1), columns=['Average']).sort_values(by='Average').reset_index()

#plot
ax=sns.barplot(x='Average', y='index',data=spending, palette='Blues')
plt.ylabel('Amount spent on...')

#add text labels for each bar's value
for p,q in zip(ax.patches, spending['Average']):
    ax.text(x=q+40,
           y=p.get_y()+0.5,
           s=q,
           ha='center');

### Which channels are underperforming?
* Channels: The avereage customer...
    * Accepted less than 1 advertising campaign
    * Made 2 deals purchases, 2 catalog purchases, 4 web purchases, and 5 store purchases
    * Averaged 14 total purchases
    * Visited the website 5 times
* Underperforming channels:
    * Advertising campaigns
    * Followed by deals, and catalog

In [99]:
channels = pd.DataFrame(round(df[channel_cols].mean(), 1), columns=['Average']).sort_values(by='Average').reset_index()

# plot
ax = sns.barplot(x='Average', y='index', data=channels, palette='Blues')
plt.ylabel('Number of...')

## add text labels for each bar's value
for p,q in zip(ax.patches, channels['Average']):
    ax.text(x=q+0.8,
            y=p.get_y()+0.5,
            s=q,
            ha="center") ;

## Conclusion
#### Recall the overall goal:
You're a marketing analyst and you've been told by the Chief Marketing Officer that recent marketing campaigns have not been as effective as they were expected to be. You need to analyze the data set to understand this problem and propose data-driven solutions...

#### Summary of actionable findings to improve advertising campaign success:
* The most successful advertising campaign was the most recent campaign (column name: Response), and was particularly successful in Mexico (>60% acceptance rate!)
    * Suggested action: Conduct future advertising campaigns using the same model recently implemented in Mexico.
* Advertising campaign acceptance is positively correlated with income and negatively correlated with having kids/teens
    * Suggested action: Create two streams of targeted advertising campaigns, one aimed at high-income individuals without kids/teens and another aimed at lower-income individuals with kids/teens
* The most successful products are wines and meats (i.e. the average customer spent the most on these items)
    * Suggested action: Focus advertising campaigns on boosting sales of the less popular items
* The underperforming channels are deals and catalog purchases (i.e. the average customer made the fewest purchases via these channels)
* The best performing channels are web and store purchases (i.e. the average customer made the most purchases via these channels)
    * Suggested action: Focus advertising campaigns on the more successful channels, to reach more customers